In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum

import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
spark = SparkSession.builder.appName("Appname").getOrCreate()

In [6]:
df = spark.read.csv("D:\\Jupiter\\cars.csv", header=True, inferSchema=True, sep=';')
df.head()

Row(Car='Chevrolet Chevelle Malibu', MPG=18.0, Cylinders=8, Displacement=307.0, Horsepower=130.0, Weight=Decimal('3504'), Acceleration=12.0, Model=70, Origin='US')

In [7]:
df.show(n=5, truncate=False)

+-------------------------+----+---------+------------+----------+------+------------+-----+------+
|Car                      |MPG |Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|
+-------------------------+----+---------+------------+----------+------+------------+-----+------+
|Chevrolet Chevelle Malibu|18.0|8        |307.0       |130.0     |3504  |12.0        |70   |US    |
|Buick Skylark 320        |15.0|8        |350.0       |165.0     |3693  |11.5        |70   |US    |
|Plymouth Satellite       |18.0|8        |318.0       |150.0     |3436  |11.0        |70   |US    |
|AMC Rebel SST            |16.0|8        |304.0       |150.0     |3433  |12.0        |70   |US    |
|Ford Torino              |17.0|8        |302.0       |140.0     |3449  |10.5        |70   |US    |
+-------------------------+----+---------+------------+----------+------+------------+-----+------+
only showing top 5 rows



In [8]:
df.limit(5)

DataFrame[Car: string, MPG: double, Cylinders: int, Displacement: double, Horsepower: double, Weight: decimal(4,0), Acceleration: double, Model: int, Origin: string]

In [9]:
df.columns

['Car',
 'MPG',
 'Cylinders',
 'Displacement',
 'Horsepower',
 'Weight',
 'Acceleration',
 'Model',
 'Origin']

In [15]:
df.dtypes

[('Car', 'string'),
 ('MPG', 'double'),
 ('Cylinders', 'int'),
 ('Displacement', 'double'),
 ('Horsepower', 'double'),
 ('Weight', 'decimal(4,0)'),
 ('Acceleration', 'double'),
 ('Model', 'int'),
 ('Origin', 'string')]

In [10]:
df.printSchema()

root
 |-- Car: string (nullable = true)
 |-- MPG: double (nullable = true)
 |-- Cylinders: integer (nullable = true)
 |-- Displacement: double (nullable = true)
 |-- Horsepower: double (nullable = true)
 |-- Weight: decimal(4,0) (nullable = true)
 |-- Acceleration: double (nullable = true)
 |-- Model: integer (nullable = true)
 |-- Origin: string (nullable = true)



In [17]:
df.select("Car").show()

+--------------------+
|                 Car|
+--------------------+
|Chevrolet Chevell...|
|   Buick Skylark 320|
|  Plymouth Satellite|
|       AMC Rebel SST|
|         Ford Torino|
|    Ford Galaxie 500|
|    Chevrolet Impala|
|   Plymouth Fury iii|
|    Pontiac Catalina|
|  AMC Ambassador DPL|
|Citroen DS-21 Pallas|
|Chevrolet Chevell...|
|    Ford Torino (sw)|
|Plymouth Satellit...|
|  AMC Rebel SST (sw)|
| Dodge Challenger SE|
|  Plymouth 'Cuda 340|
|Ford Mustang Boss...|
|Chevrolet Monte C...|
|Buick Estate Wago...|
+--------------------+
only showing top 20 rows



In [11]:
df.select(col('car')).show(truncate=False) #We have imported the 'col' from pyspark initially

+--------------------------------+
|car                             |
+--------------------------------+
|Chevrolet Chevelle Malibu       |
|Buick Skylark 320               |
|Plymouth Satellite              |
|AMC Rebel SST                   |
|Ford Torino                     |
|Ford Galaxie 500                |
|Chevrolet Impala                |
|Plymouth Fury iii               |
|Pontiac Catalina                |
|AMC Ambassador DPL              |
|Citroen DS-21 Pallas            |
|Chevrolet Chevelle Concours (sw)|
|Ford Torino (sw)                |
|Plymouth Satellite (sw)         |
|AMC Rebel SST (sw)              |
|Dodge Challenger SE             |
|Plymouth 'Cuda 340              |
|Ford Mustang Boss 302           |
|Chevrolet Monte Carlo           |
|Buick Estate Wagon (sw)         |
+--------------------------------+
only showing top 20 rows



# Selecting columns & multiple columns

In [13]:
df.select(['Car', 'Cylinders']).show()

+--------------------+---------+
|                 Car|Cylinders|
+--------------------+---------+
|Chevrolet Chevell...|        8|
|   Buick Skylark 320|        8|
|  Plymouth Satellite|        8|
|       AMC Rebel SST|        8|
|         Ford Torino|        8|
|    Ford Galaxie 500|        8|
|    Chevrolet Impala|        8|
|   Plymouth Fury iii|        8|
|    Pontiac Catalina|        8|
|  AMC Ambassador DPL|        8|
|Citroen DS-21 Pallas|        4|
|Chevrolet Chevell...|        8|
|    Ford Torino (sw)|        8|
|Plymouth Satellit...|        8|
|  AMC Rebel SST (sw)|        8|
| Dodge Challenger SE|        8|
|  Plymouth 'Cuda 340|        8|
|Ford Mustang Boss...|        8|
|Chevrolet Monte C...|        8|
|Buick Estate Wago...|        8|
+--------------------+---------+
only showing top 20 rows



In [14]:
from pyspark.sql.functions import lit
df.withColumn('first_column', lit(1)).show()

+--------------------+----+---------+------------+----------+------+------------+-----+------+------------+
|                 Car| MPG|Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|first_column|
+--------------------+----+---------+------------+----------+------+------------+-----+------+------------+
|Chevrolet Chevell...|18.0|        8|       307.0|     130.0|  3504|        12.0|   70|    US|           1|
|   Buick Skylark 320|15.0|        8|       350.0|     165.0|  3693|        11.5|   70|    US|           1|
|  Plymouth Satellite|18.0|        8|       318.0|     150.0|  3436|        11.0|   70|    US|           1|
|       AMC Rebel SST|16.0|        8|       304.0|     150.0|  3433|        12.0|   70|    US|           1|
|         Ford Torino|17.0|        8|       302.0|     140.0|  3449|        10.5|   70|    US|           1|
|    Ford Galaxie 500|15.0|        8|       429.0|     198.0|  4341|        10.0|   70|    US|           1|
|    Chevrolet Impala|14.0| 

# Adding columns

In [15]:
df.withColumn('second_column', lit(2))\
  .withColumn('Third_column', lit(2)).show()

+--------------------+----+---------+------------+----------+------+------------+-----+------+-------------+------------+
|                 Car| MPG|Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|second_column|Third_column|
+--------------------+----+---------+------------+----------+------+------------+-----+------+-------------+------------+
|Chevrolet Chevell...|18.0|        8|       307.0|     130.0|  3504|        12.0|   70|    US|            2|           2|
|   Buick Skylark 320|15.0|        8|       350.0|     165.0|  3693|        11.5|   70|    US|            2|           2|
|  Plymouth Satellite|18.0|        8|       318.0|     150.0|  3436|        11.0|   70|    US|            2|           2|
|       AMC Rebel SST|16.0|        8|       304.0|     150.0|  3433|        12.0|   70|    US|            2|           2|
|         Ford Torino|17.0|        8|       302.0|     140.0|  3449|        10.5|   70|    US|            2|           2|
|    Ford Galaxie 500|15

In [16]:
df.show()

+--------------------+----+---------+------------+----------+------+------------+-----+------+
|                 Car| MPG|Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|
+--------------------+----+---------+------------+----------+------+------------+-----+------+
|Chevrolet Chevell...|18.0|        8|       307.0|     130.0|  3504|        12.0|   70|    US|
|   Buick Skylark 320|15.0|        8|       350.0|     165.0|  3693|        11.5|   70|    US|
|  Plymouth Satellite|18.0|        8|       318.0|     150.0|  3436|        11.0|   70|    US|
|       AMC Rebel SST|16.0|        8|       304.0|     150.0|  3433|        12.0|   70|    US|
|         Ford Torino|17.0|        8|       302.0|     140.0|  3449|        10.5|   70|    US|
|    Ford Galaxie 500|15.0|        8|       429.0|     198.0|  4341|        10.0|   70|    US|
|    Chevrolet Impala|14.0|        8|       454.0|     220.0|  4354|         9.0|   70|    US|
|   Plymouth Fury iii|14.0|        8|       440.0|

# Adding columns using concat

In [18]:
from pyspark.sql.functions import concat
df.withColumn("concat_column", concat(col('Car'), lit(" "), col("Model"))).show()

+--------------------+----+---------+------------+----------+------+------------+-----+------+--------------------+
|                 Car| MPG|Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|       concat_column|
+--------------------+----+---------+------------+----------+------+------------+-----+------+--------------------+
|Chevrolet Chevell...|18.0|        8|       307.0|     130.0|  3504|        12.0|   70|    US|Chevrolet Chevell...|
|   Buick Skylark 320|15.0|        8|       350.0|     165.0|  3693|        11.5|   70|    US|Buick Skylark 320 70|
|  Plymouth Satellite|18.0|        8|       318.0|     150.0|  3436|        11.0|   70|    US|Plymouth Satellit...|
|       AMC Rebel SST|16.0|        8|       304.0|     150.0|  3433|        12.0|   70|    US|    AMC Rebel SST 70|
|         Ford Torino|17.0|        8|       302.0|     140.0|  3449|        10.5|   70|    US|      Ford Torino 70|
|    Ford Galaxie 500|15.0|        8|       429.0|     198.0|  4341|    

# Renaming Columns

In [19]:
df.withColumnRenamed("Horsepower", "H_power").show()

+--------------------+----+---------+------------+-------+------+------------+-----+------+
|                 Car| MPG|Cylinders|Displacement|H_power|Weight|Acceleration|Model|Origin|
+--------------------+----+---------+------------+-------+------+------------+-----+------+
|Chevrolet Chevell...|18.0|        8|       307.0|  130.0|  3504|        12.0|   70|    US|
|   Buick Skylark 320|15.0|        8|       350.0|  165.0|  3693|        11.5|   70|    US|
|  Plymouth Satellite|18.0|        8|       318.0|  150.0|  3436|        11.0|   70|    US|
|       AMC Rebel SST|16.0|        8|       304.0|  150.0|  3433|        12.0|   70|    US|
|         Ford Torino|17.0|        8|       302.0|  140.0|  3449|        10.5|   70|    US|
|    Ford Galaxie 500|15.0|        8|       429.0|  198.0|  4341|        10.0|   70|    US|
|    Chevrolet Impala|14.0|        8|       454.0|  220.0|  4354|         9.0|   70|    US|
|   Plymouth Fury iii|14.0|        8|       440.0|  215.0|  4312|         8.5|  

# Grouping By Columns

In [20]:
df.groupBy("Origin").count().show()

+------+-----+
|Origin|count|
+------+-----+
|Europe|   73|
|    US|  254|
| Japan|   79|
+------+-----+



In [22]:
df.groupBy("Origin", "Model").count().show()

+------+-----+-----+
|Origin|Model|count|
+------+-----+-----+
|Europe|   71|    5|
|Europe|   80|    9|
|Europe|   79|    4|
| Japan|   75|    4|
|    US|   72|   18|
|    US|   80|    7|
|Europe|   74|    6|
| Japan|   79|    2|
|Europe|   76|    8|
|    US|   75|   20|
| Japan|   77|    6|
|    US|   82|   20|
| Japan|   80|   13|
| Japan|   78|    8|
|    US|   78|   22|
|Europe|   75|    6|
|    US|   71|   20|
|    US|   77|   18|
| Japan|   70|    2|
| Japan|   71|    4|
+------+-----+-----+
only showing top 20 rows



# Adding a temp column

In [23]:
df = df.withColumn("Temp_column", lit(1))\
       .withColumn("Temp_column1", lit(2))\
       .withColumn("Temp_column3", lit(3))
print(df.show())

+--------------------+----+---------+------------+----------+------+------------+-----+------+-----------+------------+------------+
|                 Car| MPG|Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|Temp_column|Temp_column1|Temp_column3|
+--------------------+----+---------+------------+----------+------+------------+-----+------+-----------+------------+------------+
|Chevrolet Chevell...|18.0|        8|       307.0|     130.0|  3504|        12.0|   70|    US|          1|           2|           3|
|   Buick Skylark 320|15.0|        8|       350.0|     165.0|  3693|        11.5|   70|    US|          1|           2|           3|
|  Plymouth Satellite|18.0|        8|       318.0|     150.0|  3436|        11.0|   70|    US|          1|           2|           3|
|       AMC Rebel SST|16.0|        8|       304.0|     150.0|  3433|        12.0|   70|    US|          1|           2|           3|
|         Ford Torino|17.0|        8|       302.0|     140.0|  3449| 

# Removing Columns

In [24]:
df.show() #Had to add this to the variable df so this drop reflects in the next table

+--------------------+----+---------+------------+----------+------+------------+-----+------+-----------+------------+------------+
|                 Car| MPG|Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|Temp_column|Temp_column1|Temp_column3|
+--------------------+----+---------+------------+----------+------+------------+-----+------+-----------+------------+------------+
|Chevrolet Chevell...|18.0|        8|       307.0|     130.0|  3504|        12.0|   70|    US|          1|           2|           3|
|   Buick Skylark 320|15.0|        8|       350.0|     165.0|  3693|        11.5|   70|    US|          1|           2|           3|
|  Plymouth Satellite|18.0|        8|       318.0|     150.0|  3436|        11.0|   70|    US|          1|           2|           3|
|       AMC Rebel SST|16.0|        8|       304.0|     150.0|  3433|        12.0|   70|    US|          1|           2|           3|
|         Ford Torino|17.0|        8|       302.0|     140.0|  3449| 

In [25]:
df = df.drop("Temp_column", "Temp_column1")
print(df.show())


+--------------------+----+---------+------------+----------+------+------------+-----+------+------------+
|                 Car| MPG|Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|Temp_column3|
+--------------------+----+---------+------------+----------+------+------------+-----+------+------------+
|Chevrolet Chevell...|18.0|        8|       307.0|     130.0|  3504|        12.0|   70|    US|           3|
|   Buick Skylark 320|15.0|        8|       350.0|     165.0|  3693|        11.5|   70|    US|           3|
|  Plymouth Satellite|18.0|        8|       318.0|     150.0|  3436|        11.0|   70|    US|           3|
|       AMC Rebel SST|16.0|        8|       304.0|     150.0|  3433|        12.0|   70|    US|           3|
|         Ford Torino|17.0|        8|       302.0|     140.0|  3449|        10.5|   70|    US|           3|
|    Ford Galaxie 500|15.0|        8|       429.0|     198.0|  4341|        10.0|   70|    US|           3|
|    Chevrolet Impala|14.0| 

# Filtering Rows

In [26]:
total_count = df.count()
print("TOTAL RECORD COUNT: " + str(total_count))
europe_filtered_count = df.filter(col('Origin')=='Europe').count()
print("EUROPE FILTERED RECORD COUNT: " + str(europe_filtered_count))
df.filter(col('Origin')=='Europe').show(truncate=False)

TOTAL RECORD COUNT: 406
EUROPE FILTERED RECORD COUNT: 73
+----------------------------+----+---------+------------+----------+------+------------+-----+------+------------+
|Car                         |MPG |Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|Temp_column3|
+----------------------------+----+---------+------------+----------+------+------------+-----+------+------------+
|Citroen DS-21 Pallas        |0.0 |4        |133.0       |115.0     |3090  |17.5        |70   |Europe|3           |
|Volkswagen 1131 Deluxe Sedan|26.0|4        |97.0        |46.0      |1835  |20.5        |70   |Europe|3           |
|Peugeot 504                 |25.0|4        |110.0       |87.0      |2672  |17.5        |70   |Europe|3           |
|Audi 100 LS                 |24.0|4        |107.0       |90.0      |2430  |14.5        |70   |Europe|3           |
|Saab 99e                    |25.0|4        |104.0       |95.0      |2375  |17.5        |70   |Europe|3           |
|BMW 2002      

In [27]:
total_count = df.count()
print("TOTAL RECORD COUNT: " + str(total_count))
europe_filtered_count = df.filter((col('Origin')=='Europe') &
                                  (col('Cylinders')==4)).count() # Two conditions added here
print("EUROPE FILTERED RECORD COUNT: " + str(europe_filtered_count))
df.filter(col('Origin')=='Europe').show(truncate=False)

TOTAL RECORD COUNT: 406
EUROPE FILTERED RECORD COUNT: 66
+----------------------------+----+---------+------------+----------+------+------------+-----+------+------------+
|Car                         |MPG |Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|Temp_column3|
+----------------------------+----+---------+------------+----------+------+------------+-----+------+------------+
|Citroen DS-21 Pallas        |0.0 |4        |133.0       |115.0     |3090  |17.5        |70   |Europe|3           |
|Volkswagen 1131 Deluxe Sedan|26.0|4        |97.0        |46.0      |1835  |20.5        |70   |Europe|3           |
|Peugeot 504                 |25.0|4        |110.0       |87.0      |2672  |17.5        |70   |Europe|3           |
|Audi 100 LS                 |24.0|4        |107.0       |90.0      |2430  |14.5        |70   |Europe|3           |
|Saab 99e                    |25.0|4        |104.0       |95.0      |2375  |17.5        |70   |Europe|3           |
|BMW 2002      

# Unique rows

In [28]:
df.select("Cylinders").distinct().show()

+---------+
|Cylinders|
+---------+
|        6|
|        3|
|        5|
|        4|
|        8|
+---------+



# Unique rows with multiple columns

In [29]:
df.select("Cylinders", "Acceleration").distinct().show()

+---------+------------+
|Cylinders|Acceleration|
+---------+------------+
|        6|        15.5|
|        4|        15.6|
|        8|        12.8|
|        8|        17.4|
|        6|        16.5|
|        8|        15.5|
|        5|        19.9|
|        4|        20.4|
|        6|        14.5|
|        6|        16.7|
|        8|        22.2|
|        4|        19.2|
|        4|        12.8|
|        4|        15.3|
|        6|        11.3|
|        4|        18.3|
|        6|        17.1|
|        8|        19.0|
|        4|        13.5|
|        6|        13.0|
+---------+------------+
only showing top 20 rows



# Using Order by, this will be ascending by default

In [30]:
df.orderBy("Cylinders").show()

+--------------------+----+---------+------------+----------+------+------------+-----+------+------------+
|                 Car| MPG|Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|Temp_column3|
+--------------------+----+---------+------------+----------+------+------------+-----+------+------------+
|     Mazda RX2 Coupe|19.0|        3|        70.0|      97.0|  2330|        13.5|   72| Japan|           3|
|           Mazda RX3|18.0|        3|        70.0|      90.0|  2124|        13.5|   73| Japan|           3|
|          Mazda RX-4|21.5|        3|        80.0|     110.0|  2720|        13.5|   77| Japan|           3|
|       Mazda RX-7 GS|23.7|        3|        70.0|     100.0|  2420|        12.5|   80| Japan|           3|
|     Datsun 510 (sw)|28.0|        4|        97.0|      92.0|  2288|        17.0|   72| Japan|           3|
|  Mercury Capri 2000|23.0|        4|       122.0|      86.0|  2220|        14.0|   71|    US|           3|
| Chevrolet Vega (sw)|22.0| 

In [31]:
df.orderBy("Cylinders", ascending=False).show(n=5) #revoking the ascending function

+--------------------+----+---------+------------+----------+------+------------+-----+------+------------+
|                 Car| MPG|Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|Temp_column3|
+--------------------+----+---------+------------+----------+------+------------+-----+------+------------+
|Chevrolet Chevell...|18.0|        8|       307.0|     130.0|  3504|        12.0|   70|    US|           3|
|   Buick Skylark 320|15.0|        8|       350.0|     165.0|  3693|        11.5|   70|    US|           3|
|  Plymouth Satellite|18.0|        8|       318.0|     150.0|  3436|        11.0|   70|    US|           3|
|       AMC Rebel SST|16.0|        8|       304.0|     150.0|  3433|        12.0|   70|    US|           3|
|         Ford Torino|17.0|        8|       302.0|     140.0|  3449|        10.5|   70|    US|           3|
+--------------------+----+---------+------------+----------+------+------------+-----+------+------------+
only showing top 5 rows



# Using groupBy aand orderBy together

In [33]:
 df.groupBy("Origin").count().orderBy('count').show()

+------+-----+
|Origin|count|
+------+-----+
|Europe|   73|
| Japan|   79|
|    US|  254|
+------+-----+



# Union Dataframes

You will see three main methods for performing union of dataframes. It is important to know the difference between them and which one is preferred:

#union() – It is used to merge two DataFrames of the same structure/schema. If schemas are not the same, it returns an error.

#unionAll() – This function is deprecated since Spark 2.0.0, and replaced with union()

#unionByName() - This function is used to merge two dataframes based on column name.

In [34]:
ECARS = df.filter((col("Origin")=="Europe") & (col("Cylinders")== 4)) #with multiple conditions
JCARS = df.filter((col("Origin")=="Japan") & (col("Cylinders")== 5)) ##with multiple conditions
print("European Cars : " + str(ECARS.count()))
print("Japanese Cars: " + str(JCARS.count()))
print("After union :" +str(ECARS.union(JCARS).count()))

European Cars : 66
Japanese Cars: 0
After union :66


# Another easy example of union join

In [35]:
df1 =df.filter(col("Origin")== "Japan")
df2 = df.filter(col("weight")== 2330)
df.union(df2).show()

+--------------------+----+---------+------------+----------+------+------------+-----+------+------------+
|                 Car| MPG|Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|Temp_column3|
+--------------------+----+---------+------------+----------+------+------------+-----+------+------------+
|Chevrolet Chevell...|18.0|        8|       307.0|     130.0|  3504|        12.0|   70|    US|           3|
|   Buick Skylark 320|15.0|        8|       350.0|     165.0|  3693|        11.5|   70|    US|           3|
|  Plymouth Satellite|18.0|        8|       318.0|     150.0|  3436|        11.0|   70|    US|           3|
|       AMC Rebel SST|16.0|        8|       304.0|     150.0|  3433|        12.0|   70|    US|           3|
|         Ford Torino|17.0|        8|       302.0|     140.0|  3449|        10.5|   70|    US|           3|
|    Ford Galaxie 500|15.0|        8|       429.0|     198.0|  4341|        10.0|   70|    US|           3|
|    Chevrolet Impala|14.0| 

# CASE 1: Union When columns are not in order

# Creating two dataframes with jumbled columns

In [36]:
df1 = spark.createDataFrame([["Apple", "Ball", "cat"]], ["col0", "col1", "col2"])
df2 = spark.createDataFrame([["Ant", "Baseball", "Caterpillar"]], ["col1", "col2", "col0"])
df1.unionByName(df2).show()

+-----------+----+--------+
|       col0|col1|    col2|
+-----------+----+--------+
|      Apple|Ball|     cat|
|Caterpillar| Ant|Baseball|
+-----------+----+--------+



In [87]:
df.show()

+--------------------+----+---------+------------+----------+------+------------+-----+------+------------+
|                 Car| MPG|Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|Temp_column3|
+--------------------+----+---------+------------+----------+------+------------+-----+------+------------+
|Chevrolet Chevell...|18.0|        8|       307.0|     130.0|  3504|        12.0|   70|    US|           3|
|   Buick Skylark 320|15.0|        8|       350.0|     165.0|  3693|        11.5|   70|    US|           3|
|  Plymouth Satellite|18.0|        8|       318.0|     150.0|  3436|        11.0|   70|    US|           3|
|       AMC Rebel SST|16.0|        8|       304.0|     150.0|  3433|        12.0|   70|    US|           3|
|         Ford Torino|17.0|        8|       302.0|     140.0|  3449|        10.5|   70|    US|           3|
|    Ford Galaxie 500|15.0|        8|       429.0|     198.0|  4341|        10.0|   70|    US|           3|
|    Chevrolet Impala|14.0| 

# Functions in Pyspark

In [37]:
from pyspark.sql import functions 

In [38]:
print(dir(functions))

['Any', 'ArrayType', 'Callable', 'Column', 'DataFrame', 'DataType', 'Dict', 'Iterable', 'JVMView', 'List', 'Optional', 'PandasUDFType', 'PySparkTypeError', 'PySparkValueError', 'SparkContext', 'StringType', 'StructType', 'TYPE_CHECKING', 'Tuple', 'Type', 'Union', 'UserDefinedFunction', 'UserDefinedTableFunction', 'ValuesView', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '_create_column_from_literal', '_create_lambda', '_create_py_udf', '_create_py_udtf', '_from_numpy_type', '_get_jvm_function', '_get_lambda_parameters', '_invoke_binary_math_function', '_invoke_function', '_invoke_function_over_columns', '_invoke_function_over_seq_of_columns', '_invoke_higher_order_function', '_options_to_str', '_test', '_to_java_column', '_to_seq', '_unresolved_named_lambda_variable', 'abs', 'acos', 'acosh', 'add_months', 'aes_decrypt', 'aes_encrypt', 'aggregate', 'any_value', 'approxCountDistinct', 'approx_count_distinct', 'approx_percentil

In [2]:
from pyspark.sql.functions import col,lower, upper, substring
help(substring)

Help on function substring in module pyspark.sql.functions:

substring(str: 'ColumnOrName', pos: int, len: int) -> pyspark.sql.column.Column
    Substring starts at `pos` and is of length `len` when str is String type or
    returns the slice of byte array that starts at `pos` in byte and is of length `len`
    when str is Binary type.
    
    .. versionadded:: 1.5.0
    
    .. versionchanged:: 3.4.0
        Supports Spark Connect.
    
    Notes
    -----
    The position is not zero based, but 1 based index.
    
    Parameters
    ----------
    str : :class:`~pyspark.sql.Column` or str
        target column to work on.
    pos : int
        starting position in str.
    len : int
        length of chars.
    
    Returns
    -------
    :class:`~pyspark.sql.Column`
        substring of given value.
    
    Examples
    --------
    >>> df = spark.createDataFrame([('abcd',)], ['s',])
    >>> df.select(substring(df.s, 1, 2).alias('s')).collect()
    [Row(s='ab')]



In [39]:
from pyspark.sql.functions import col,lower, upper, substring
# Prints out the details of a function
# alias is used to rename the column name in the output
df.select(col('Car'),lower(col('Car')),upper(col('Car')),substring(col('Car'),1,4).alias("concatenated value")).show(5, False)

+-------------------------+-------------------------+-------------------------+------------------+
|Car                      |lower(Car)               |upper(Car)               |concatenated value|
+-------------------------+-------------------------+-------------------------+------------------+
|Chevrolet Chevelle Malibu|chevrolet chevelle malibu|CHEVROLET CHEVELLE MALIBU|Chev              |
|Buick Skylark 320        |buick skylark 320        |BUICK SKYLARK 320        |Buic              |
|Plymouth Satellite       |plymouth satellite       |PLYMOUTH SATELLITE       |Plym              |
|AMC Rebel SST            |amc rebel sst            |AMC REBEL SST            |AMC               |
|Ford Torino              |ford torino              |FORD TORINO              |Ford              |
+-------------------------+-------------------------+-------------------------+------------------+
only showing top 5 rows



In [40]:
df.select(col("Car"), col("Model"), concat(col("Car"), lit(" "), (col("Model")))).show(5, False)

+-------------------------+-----+----------------------------+
|Car                      |Model|concat(Car,  , Model)       |
+-------------------------+-----+----------------------------+
|Chevrolet Chevelle Malibu|70   |Chevrolet Chevelle Malibu 70|
|Buick Skylark 320        |70   |Buick Skylark 320 70        |
|Plymouth Satellite       |70   |Plymouth Satellite 70       |
|AMC Rebel SST            |70   |AMC Rebel SST 70            |
|Ford Torino              |70   |Ford Torino 70              |
+-------------------------+-----+----------------------------+
only showing top 5 rows



In [41]:
from pyspark.sql.functions import min, max, lit
df.select(min(col("Model")) + lit(10), max(col('Model') + lit(100))).show()

+-----------------+------------------+
|(min(Model) + 10)|max((Model + 100))|
+-----------------+------------------+
|               80|               182|
+-----------------+------------------+



In [42]:
from pyspark.sql.functions import to_date, to_timestamp, lit
df = spark.createDataFrame([('2019-12-25 13:30:00',)], ['DOB'])
df.show()
df.printSchema()

+-------------------+
|                DOB|
+-------------------+
|2019-12-25 13:30:00|
+-------------------+

root
 |-- DOB: string (nullable = true)



In [43]:
from pyspark.sql.functions import col, to_date, to_timestamp
df = spark.createDataFrame([('2013-12-25 13:30:00',)], ['DOB'])
df = df.select(to_date(col("DOB"), 'yyyy-MM-dd HH:mm:ss').alias("Date"),
               to_timestamp(col("DOB"), 'yyyy-MM-dd HH:mm:ss').alias("Timestamp"))
df.show(truncate=False)

+----------+-------------------+
|Date      |Timestamp          |
+----------+-------------------+
|2013-12-25|2013-12-25 13:30:00|
+----------+-------------------+

